This is a quick dirty code to convert connectivity matrices to a tidy tabular format that can be used in stat tools.

In [4]:
from src.acnets.pipeline import Parcellation, ConnectivityExtractor
from pathlib import Path

kind = 'tangent'

p = Parcellation(
    atlas_name='dosenbach2010',
    bids_dir=Path('data/julia2018/'),
    fmriprep_bids_space='MNI152NLin2009cAsym',
    normalize=True
)

ce = ConnectivityExtractor(kind=kind)

ce_ds = ce.fit_transform(p.fit_transform(X=None))

df = ce_ds['connectivity'].to_dataframe().reset_index()

networks_df = ce_ds['network'].to_dataframe().reset_index()

df = df.merge(networks_df, left_on='region_src', right_on='region', how='left').rename(columns={'network': 'network_src'})
df = df.merge(networks_df, left_on='region_dst', right_on='region', how='left').rename(columns={'network': 'network_dst'})

df.drop(columns=['region_x', 'region_y'], inplace=True)
df = df[['subject', 'network_src', 'network_dst', 'region_src', 'region_dst', 'connectivity']]

df.to_csv('data/julia2018/derivatives/connectivity/dosenbach2010_{}.csv'.format(kind.replace(' ', '_')), index=False)